In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
minst = input_data.read_data_sets('MINST_data', one_hot = True)

Extracting MINST_data/train-images-idx3-ubyte.gz
Extracting MINST_data/train-labels-idx1-ubyte.gz
Extracting MINST_data/t10k-images-idx3-ubyte.gz
Extracting MINST_data/t10k-labels-idx1-ubyte.gz


&emsp;使用交互式对话，不用一次性构建整个图再运行

In [2]:
sess = tf.InteractiveSession()

&emsp;设置超参数，这里的输入是图片的一行，每行作为时序中的一帧，time_step是时序的长度

In [3]:
lr = 0.001
iteration = 20000
batch_size = 128

input_size = 28
time_step = 28
neurons = 128
classes = 10

In [4]:
xs = tf.placeholder(tf.float32, [None, time_step, input_size])
ys = tf.placeholder(tf.float32, [None, classes])

&emsp;设置权重,这里权重分为输入权重与输出权重

In [5]:
weight = {
    'in':tf.Variable(tf.random_normal([input_size, neurons])),
    'out':tf.Variable(tf.random_normal([neurons, classes]))
}
bias = {
    'in':tf.Variable(tf.constant(0.1, shape = [neurons, ])),
    'out':tf.Variable(tf.constant(0.1, shape = [classes, ]))
}

&emsp;第一次reshape中的 -1 是指该维先不计算，最后通过已确定的维度来算，这里共有X原维度为[batch_size = 128, time_step = 28, input_size = 28],也就是共有128 \* 28 \* 28个元素。reshape之后第二维为28个元素即一行像素,则第一维为 128 \* 28 \* 28 / 28 = 128 \* 28。其实是将原数据转化为了若干个28像素的输入,每个28像素即一个时序帧的输入   
&emsp;第二次reshape同理,将结果转换回输入时的格式  
&emsp;BasicLSTMCell是最简单的LSTMCell,只含一个forget门、一个input门、一个output门。state_is_tuple = True,是指以tuple形式(c_state, m_state)返回cell state,其中m_state其实就是输出。lstm_cell.zero_state用0来初始化前一状态  
&emsp;dynamic_rnn中的time_major函数决定了输出的格式,time_major = false的话输出格式为[batch_size, time_step, neurons],否则batch_size与time_step的位置互换  
&emsp;out储存了每一步的输出,是一个list,而states只保存最后一步的states,因此states[1]就是最后的输出

In [6]:
def RNN(X, W, B):
    X = tf.reshape(X, [-1, input_size])
    in_ = tf.matmul(X, W['in']) + B['in']
    in_ = tf.reshape(in_, [-1, time_step, neurons])
    
    first_lstm_cell = tf.contrib.rnn.BasicLSTMCell(neurons, forget_bias = 1.0, state_is_tuple = True)
    init_state = first_lstm_cell.zero_state(batch_size, dtype = tf.float32)
    
    out_, states_ = tf.nn.dynamic_rnn(first_lstm_cell, in_, initial_state = init_state, time_major = False)
    
    results = tf.matmul(states_[1], W['out']) + B['out']
    return results

&emsp;这里的softmax_cross_entropy_with_logits实际是将softmax与cross_entropy合为一步,因为通常softmax的损失函数就是cross_entropy

In [7]:
pred = RNN(xs, weight, bias)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = pred, labels = ys))
train = tf.train.AdamOptimizer(lr).minimize(cost)

In [8]:
correct_pred = tf.equal(tf.arg_max(pred, 1), tf.arg_max(ys, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
sess.run(tf.global_variables_initializer())
for step in range(20000):
    batch_xs, batch_ys = minst.train.next_batch(batch_size)
    batch_xs = batch_xs.reshape([batch_size, time_step, input_size])
    sess.run(train, feed_dict={xs: batch_xs, ys: batch_ys})
    if step % 20 == 0:
        print(sess.run(accuracy, feed_dict={xs: batch_xs, ys: batch_ys}))

0.171875
0.71875
0.695312
0.78125
0.859375
0.828125
0.90625
0.914062
0.882812
0.914062
0.898438
0.914062
0.929688
0.898438
0.9375
0.96875
0.9375
0.976562
0.960938
0.945312
0.960938
0.953125
0.984375
0.953125
0.96875
0.976562
0.960938
0.976562
0.953125
0.953125
0.9375
0.953125
0.992188
0.960938
0.945312
0.96875
0.976562
0.953125
0.96875
0.992188
0.945312
0.96875
0.984375
0.976562
0.992188
0.976562
0.976562
0.96875
0.96875
0.960938
0.992188
0.984375
0.976562
0.984375
0.976562
0.976562
0.984375
1.0
0.976562
0.984375
0.984375
0.976562
0.984375
1.0
0.984375
0.96875
0.976562
0.984375
1.0
0.976562
0.984375
0.960938
0.96875
0.992188
0.976562
0.992188
0.992188
0.984375
1.0
0.96875
0.96875
0.976562
0.960938
0.992188
0.976562
0.976562
0.984375
0.992188
1.0
0.992188
0.992188
0.992188
0.984375
1.0
1.0
0.992188
1.0
1.0
1.0
1.0
0.976562
0.984375
1.0
1.0
0.976562
0.992188
0.96875
1.0
0.984375
0.984375
0.96875
0.992188
0.976562
1.0
1.0
0.992188
0.984375
0.992188
0.96875
1.0
1.0
0.992188
1.0
0.992188
1.